# Reference Metrics and Coordinate Systems

## Author: Zach Etienne

## Exploring the reference metric infrastructure in NRPy, this notebook shows how to construct `ReferenceMetric` objects for different coordinate systems, inspect the associated tensors, and use the precomputation framework. It focuses on hands-on examples so that you can treat the reference metric as a black box with a clear Python interface.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, use for reference; you should be able to pick up the syntax as you follow the tutorial.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**

### NRPy Source Code for this module:
* [reference_metric.py](../edit/reference_metric.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python/NRPy-modules-for-ReferenceMetric): Initialize core Python/NRPy modules for ReferenceMetric
1. [Step 2](#Step-2:-Exploring-supported-coordinate-systems-and-rfm_dict): Exploring supported coordinate systems and rfm_dict
1. [Step 3](#Step-3:-Cartesian-reference-metric:-xx,-xx_to_Cart,-ghatDD,-detgammahat): Cartesian reference metric: `xx`, `xx_to_Cart`, `ghatDD`, `detgammahat`
1. [Step 4](#Step-4:-Spherical-reference-metrics-and-unit-vectors): Spherical reference metrics and unit vectors
1. [Step 5](#Step-5:-Transformations-between-charts:-xx_to_Cart,-Cart_to_xx,-xxSph,-and-Jacobians): Transformations between charts: `xx_to_Cart`, `Cart_to_xx`, `xxSph`, and Jacobians
1. [Step 6](#Step-6:-Reference-metric-tensors,-rescaling-vectors,-and-determinants): Reference metric tensors, rescaling vectors, and determinants
1. [Step 7](#Step-7:-Derivatives-and-Christoffel-symbols-from-the-reference-metric): Derivatives and Christoffel symbols from the reference metric
1. [Step 8](#Step-8:-Precomputing-expensive-functions-with-_rfm_precompute): Precomputing expensive functions with `_rfm_precompute`
1. [Step 9](#Step-9:-Advanced-coordinate-maps:-sinh-families,-NewtonRaphson-inversions,-and-validation): Advanced coordinate maps: sinh families, NewtonRaphson inversions, and validation

# Step 1: Initialize core Python/NRPy modules for ReferenceMetric
### \[Back to [top](#Table-of-Contents)\]

The reference metric infrastructure lives in `nrpy.reference_metric`. In practice you will almost always access it through the global dictionary-like object `reference_metric`, which lazily constructs and caches `ReferenceMetric` instances keyed by coordinate system name.

In this step we import the minimum set of modules that will be reused throughout the notebook.

In [1]:
# Step 1: Initialize core Python/NRPy modules for ReferenceMetric
import sympy as sp

import nrpy.reference_metric as rfm
import nrpy.params as par

# Convenience handle for the global cache of ReferenceMetric objects:
reference_metric = rfm.reference_metric

print("Type of reference_metric:", type(reference_metric))

Type of reference_metric: <class 'nrpy.reference_metric.rfm_dict'>


# Step 2: Exploring supported coordinate systems and rfm_dict
### \[Back to [top](#Table-of-Contents)\]

The global object `reference_metric` is an instance of the custom class `rfm_dict`, which behaves like a Python dictionary:

* Keys are strings such as `"Cartesian"` or `"Spherical"`.
* Values are `ReferenceMetric` objects.
* The first time you request `reference_metric["SomeCoordSystem"]`, the corresponding `ReferenceMetric` is constructed automatically.

The module also exposes two helpful lists:

* `supported_CoordSystems`: everything that `ReferenceMetric` knows how to construct.
* `unittest_CoordSystems`: a small subset used in automated tests.

Let us inspect these objects and build a couple of reference metrics.

In [2]:
# Step 2: Exploring supported coordinate systems and rfm_dict

print("Supported CoordSystems:")
for name in rfm.supported_CoordSystems:
    print(" -", name)

print("\nCoordSystems used in unit tests:")
for name in rfm.unittest_CoordSystems:
    print(" -", name)

# Build (or fetch from the cache) some commonly used reference metrics:
rfm_cart = reference_metric["Cartesian"]
rfm_sph = reference_metric["Spherical"]

print("\nCreated ReferenceMetric instances:")
print(" - Cartesian CoordSystem:", rfm_cart.CoordSystem)
print(" - Spherical CoordSystem:", rfm_sph.CoordSystem)

# The first access to a CoordSystem automatically also creates its _rfm_precompute partner:
rfm_cart_pre = reference_metric["Cartesian_rfm_precompute"]
print(" - Cartesian_rfm_precompute CoordSystem:", rfm_cart_pre.CoordSystem)

Supported CoordSystems:
 - Spherical
 - SinhSpherical
 - SinhSphericalv2n2
 - Cartesian
 - SinhCartesian
 - Cylindrical
 - SinhCylindrical
 - SinhCylindricalv2n2
 - SymTP
 - SinhSymTP
 - LWedgeHSinhSph
 - UWedgeHSinhSph
 - RingHoleySinhSpherical
 - HoleySinhSpherical

CoordSystems used in unit tests:
 - SinhSymTP
 - HoleySinhSpherical
 - Cartesian
 - SinhCylindricalv2n2
Setting up reference_metric[Cartesian]...
Setting up reference_metric[Spherical]...

Created ReferenceMetric instances:
 - Cartesian CoordSystem: Cartesian
 - Spherical CoordSystem: Spherical
 - Cartesian_rfm_precompute CoordSystem: Cartesian


# Step 3: Cartesian reference metric: xx, xx_to_Cart, ghatDD, detgammahat
### \[Back to [top](#Table-of-Contents)\]

Each `ReferenceMetric` instance stores a complete description of a chosen coordinate system:

* `xx`: the abstract grid coordinates `xx[0]`, `xx[1]`, `xx[2]`.
* `xx_to_Cart`: the map from grid coordinates to Cartesian coordinates `(Cartx, Carty, Cartz)`.
* `Cart_to_xx`: the inverse map from Cartesian coordinates back to the grid coordinates.
* `xxSph`: the same point expressed in spherical coordinates `(r, theta, phi)`.

In addition, the reference metric tensors are stored in:

* `ghatDD[i][j]`: the covariant metric components.
* `ghatUU[i][j]`: the contravariant components (the matrix inverse of `ghatDD`).
* `detgammahat`: the determinant of `ghatDD`.

For the pure Cartesian coordinate system the reference metric is just the flat Euclidean metric, so you should expect `ghatDD` to be the identity matrix and `detgammahat` to be $1$.

Let us confirm this directly from the Python interface.

In [3]:
# Step 3: Inspect Cartesian reference metric data

rfm_cart = reference_metric["Cartesian"]

print("Grid coordinates xx:")
print(rfm_cart.xx)

print("\nMap xx_to_Cart (Cartesian coordinates expressed in terms of xx):")
for comp in rfm_cart.xx_to_Cart:
    print(" ", comp)

print("\nInverse map Cart_to_xx (xx expressed in terms of Cartx, Carty, Cartz):")
for comp in rfm_cart.Cart_to_xx:
    print(" ", comp)

print("\nCovariant metric ghatDD for Cartesian coordinates:")
for row in rfm_cart.ghatDD:
    print(" ", row)

print("\nInverse metric ghatUU:")
for row in rfm_cart.ghatUU:
    print(" ", row)

print("\nDeterminant detgammahat:")
print(rfm_cart.detgammahat)

# Sanity check: ghatUU * ghatDD should give the identity matrix.
gDD = sp.Matrix(rfm_cart.ghatDD)
gUU = sp.Matrix(rfm_cart.ghatUU)
print("\nCheck gUU * gDD:")
print(sp.simplify(gUU * gDD))

Grid coordinates xx:
[xx0, xx1, xx2]

Map xx_to_Cart (Cartesian coordinates expressed in terms of xx):
  xx0
  xx1
  xx2

Inverse map Cart_to_xx (xx expressed in terms of Cartx, Carty, Cartz):
  Cartx
  Carty
  Cartz

Covariant metric ghatDD for Cartesian coordinates:
  [1, 0, 0]
  [0, 1, 0]
  [0, 0, 1]

Inverse metric ghatUU:
  [1, 0, 0]
  [0, 1, 0]
  [0, 0, 1]

Determinant detgammahat:
1

Check gUU * gDD:
Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])


# Step 4: Spherical reference metrics and unit vectors
### \[Back to [top](#Table-of-Contents)\]

Spherical coordinates are more interesting: the metric is no longer diagonal with all entries equal to $1$, and the grid coordinates `(xx[0], xx[1], xx[2])` naturally align with the usual spherical coordinates `(r, theta, phi)`.

The `ReferenceMetric` object for `"Spherical"` gives you:

* `xxSph`: aliases for `(r, theta, phi)` written as SymPy expressions in terms of `xx`.
* `scalefactor_orthog`: the orthogonal scale factors along each coordinate line.
* `UnitVectors`: a $3 \times 3$ matrix whose rows describe the orthonormal basis vectors associated with the coordinate directions, written in Cartesian components.
* `radial_like_dirns`: a list of indices indicating which coordinate directions are treated as "radial-like" by higher level code.

Let us explore these pieces.

In [4]:
# Step 4: Inspect spherical reference metric data

rfm_sph = reference_metric["Spherical"]

print("Grid coordinates xx (Spherical):")
print(rfm_sph.xx)

print("\nSpherical coordinates xxSph (r, theta, phi) in terms of xx:")
for comp in rfm_sph.xxSph:
    print(" ", comp)

print("\nOrthogonal scale factors scalefactor_orthog:")
for h in rfm_sph.scalefactor_orthog:
    print(" ", h)

print("\nUnitVectors (rows are basis vectors along r, theta, phi, written in Cartesian components):")
for row in rfm_sph.UnitVectors:
    print(" ", row)

print("\nRadial-like coordinate directions:")
print(rfm_sph.radial_like_dirns)

Grid coordinates xx (Spherical):
[xx0, xx1, xx2]

Spherical coordinates xxSph (r, theta, phi) in terms of xx:
  xx0
  xx1
  xx2

Orthogonal scale factors scalefactor_orthog:
  1
  xx0
  xx0*sin(xx1)

UnitVectors (rows are basis vectors along r, theta, phi, written in Cartesian components):
  [sin(xx1)*cos(xx2), sin(xx1)*sin(xx2), cos(xx1)]
  [cos(xx1)*cos(xx2), sin(xx2)*cos(xx1), -sin(xx1)]
  [-sin(xx2), cos(xx2), 0]

Radial-like coordinate directions:
[0]


# Step 5: Transformations between charts: xx_to_Cart, Cart_to_xx, xxSph, and Jacobians
### \[Back to [top](#Table-of-Contents)\]

For each coordinate system the `ReferenceMetric` object stores:

* Forward maps such as `xx_to_Cart` and `xxSph`.
* Backward maps such as `Cart_to_xx`.
* Jacobians of these maps, already symbolically differentiated for you:
  * `Jac_dUCart_dDrfmUD[i][j]` is $\partial x^{Cart}_i / \partial x^{rfm}_j$.
  * `Jac_dUrfm_dDCartUD[i][j]` is its inverse.
  * `Jac_dUSph_dDrfmUD` and `Jac_dUrfm_dDSphUD` play the same role for spherical coordinates.

In this step we verify that the forward and backward maps for `"Spherical"` really invert each other symbolically, and we inspect one of the Jacobians.

In [5]:
# Step 5: Verify Spherical forward/backward maps and inspect Jacobians

rfm_sph = reference_metric["Spherical"]

# Shortcuts for symbols
r_sym, th_sym, ph_sym = rfm_sph.xx
Cartx, Carty, Cartz = rfm_sph.Cartx, rfm_sph.Carty, rfm_sph.Cartz

# Forward map: xx -> Cartesian
Cartx_expr = rfm_sph.xx_to_Cart[0]
Carty_expr = rfm_sph.xx_to_Cart[1]
Cartz_expr = rfm_sph.xx_to_Cart[2]

# Substitute forward map into the backward map Cart_to_xx
r_back = rfm_sph.Cart_to_xx[0].subs({Cartx: Cartx_expr, Carty: Carty_expr, Cartz: Cartz_expr})
th_back = rfm_sph.Cart_to_xx[1].subs({Cartx: Cartx_expr, Carty: Carty_expr, Cartz: Cartz_expr})
ph_back = rfm_sph.Cart_to_xx[2].subs({Cartx: Cartx_expr, Carty: Carty_expr, Cartz: Cartz_expr})

print("Forward then backward map (symbolic):")
print(" r_back  - r_sym  =", sp.simplify(r_back - r_sym))
print(" th_back - th_sym =", sp.simplify(th_back - th_sym))
print(" ph_back - ph_sym =", sp.simplify(ph_back - ph_sym))

# Now inspect Jacobians between rfm and Cartesian coordinates.
print("\nJacobian dUCart_dDrfmUD (Cartesian components with respect to xx):")
for row in rfm_sph.Jac_dUCart_dDrfmUD:
    print(" ", row)

print("\nInverse Jacobian dUrfm_dDCartUD (xx components with respect to Cartesian):")
for row in rfm_sph.Jac_dUrfm_dDCartUD:
    print(" ", row)

# Check that the Jacobian and its inverse really multiply to the identity.
J = sp.Matrix(rfm_sph.Jac_dUCart_dDrfmUD)
Jinv = sp.Matrix(rfm_sph.Jac_dUrfm_dDCartUD)
print("\nCheck J * Jinv:")
print(sp.simplify(J * Jinv))

Forward then backward map (symbolic):
 r_back  - r_sym  = -xx0 + sqrt(xx0**2)
 th_back - th_sym = -xx1 + acos(xx0*cos(xx1)/sqrt(xx0**2))
 ph_back - ph_sym = -xx2 + atan2(xx0*sin(xx1)*sin(xx2), xx0*sin(xx1)*cos(xx2))

Jacobian dUCart_dDrfmUD (Cartesian components with respect to xx):
  [sin(xx1)*cos(xx2), xx0*cos(xx1)*cos(xx2), -xx0*sin(xx1)*sin(xx2)]
  [sin(xx1)*sin(xx2), xx0*sin(xx2)*cos(xx1), xx0*sin(xx1)*cos(xx2)]
  [cos(xx1), -xx0*sin(xx1), 0]

Inverse Jacobian dUrfm_dDCartUD (xx components with respect to Cartesian):
  [xx0**2*sin(xx1)**2*cos(xx2)/(xx0**2*sin(xx1)**3*sin(xx2)**2 + xx0**2*sin(xx1)**3*cos(xx2)**2 + xx0**2*sin(xx1)*sin(xx2)**2*cos(xx1)**2 + xx0**2*sin(xx1)*cos(xx1)**2*cos(xx2)**2), xx0**2*sin(xx1)**2*sin(xx2)/(xx0**2*sin(xx1)**3*sin(xx2)**2 + xx0**2*sin(xx1)**3*cos(xx2)**2 + xx0**2*sin(xx1)*sin(xx2)**2*cos(xx1)**2 + xx0**2*sin(xx1)*cos(xx1)**2*cos(xx2)**2), (xx0**2*sin(xx1)*sin(xx2)**2*cos(xx1) + xx0**2*sin(xx1)*cos(xx1)*cos(xx2)**2)/(xx0**2*sin(xx1)**3*sin(xx2)**2 +

# Step 6: Reference metric tensors, rescaling vectors, and determinants
### \[Back to [top](#Table-of-Contents)\]

Beyond the basic coordinate transformations, `ReferenceMetric` exposes a number of tensors that encode the geometry of the reference metric:

* `ghatDD[i][j]`: the covariant metric components $\hat{g}_{ij}$.
* `ghatUU[i][j]`: the contravariant components $\hat{g}^{ij}$.
* `detgammahat`: the determinant $\det(\hat{g})$.
* `ReU[i]`, `ReD[i]`: rescaling vectors often denoted $\mathcal{R}^i$ and $\mathcal{R}_i$.
* `ReDD[i][j]`: a rescaling matrix used when converting between physical and conformal tensors.

These quantities are particularly important in formulations such as BSSN, where tensor components are split into conformal and rescaled pieces.

Here we take a closer look at these objects for the `"Spherical"` coordinate system and confirm that `ghatUU` is indeed the matrix inverse of `ghatDD`.

In [6]:
# Step 6: Inspect tensors and rescaling vectors for Spherical coordinates

rfm_sph = reference_metric["Spherical"]

print("Covariant metric ghatDD:")
for row in rfm_sph.ghatDD:
    print(" ", row)

print("\nInverse metric ghatUU:")
for row in rfm_sph.ghatUU:
    print(" ", row)

print("\nDeterminant detgammahat:")
print(rfm_sph.detgammahat)

print("\nRescaling vectors ReU (upper) and ReD (lower):")
print(" ReU:", rfm_sph.ReU)
print(" ReD:", rfm_sph.ReD)

print("\nRescaling matrix ReDD:")
for row in rfm_sph.ReDD:
    print(" ", row)

# Verify ghatUU is the inverse of ghatDD.
gDD = sp.Matrix(rfm_sph.ghatDD)
gUU = sp.Matrix(rfm_sph.ghatUU)
identity_check = sp.simplify(gUU * gDD)
print("\nCheck gUU * gDD (should be the identity matrix):")
print(identity_check)

Covariant metric ghatDD:
  [1, 0, 0]
  [0, xx0**2, 0]
  [0, 0, xx0**2*sin(xx1)**2]

Inverse metric ghatUU:
  [1, 0, 0]
  [0, xx0**(-2), 0]
  [0, 0, 1/(xx0**2*sin(xx1)**2)]

Determinant detgammahat:
xx0**4*sin(xx1)**2

Rescaling vectors ReU (upper) and ReD (lower):
 ReU: [1, 1/xx0, 1/(xx0*sin(xx1))]
 ReD: [1, xx0, xx0*sin(xx1)]

Rescaling matrix ReDD:
  [1, xx0, xx0*sin(xx1)]
  [xx0, xx0**2, xx0**2*sin(xx1)]
  [xx0*sin(xx1), xx0**2*sin(xx1), xx0**2*sin(xx1)**2]

Check gUU * gDD (should be the identity matrix):
Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])


# Step 7: Derivatives and Christoffel symbols from the reference metric
### \[Back to [top](#Table-of-Contents)\]

Many evolution systems involve derivatives of the metric and connections built from those derivatives. `ReferenceMetric` computes these symbolically once and stores them in a variety of arrays:

* `detgammahatdD[i]`: first derivatives $\partial_i \det(\hat{g})$.
* `detgammahatdDD[i][j]`: second derivatives $\partial_i \partial_j \det(\hat{g})$.
* `ghatDDdD[i][j][k]`: first derivatives of the metric $\partial_k \hat{g}_{ij}$.
* `ghatDDdDD[i][j][k][l]`: second derivatives $\partial_l \partial_k \hat{g}_{ij}$.
* `GammahatUDD[i][j][k]`: Christoffel symbols $\hat{\Gamma}^i{}_{jk}$.
* `GammahatUDDdD[i][j][k][l]`: derivatives $\partial_l \hat{\Gamma}^i{}_{jk}$.

In this step we pick a few representative components for the `"Spherical"` coordinate system and evaluate them at a specific point. This gives a concrete feel for what these tensors look like numerically.

We will evaluate at the point $r = 2$, $\theta = \pi/4$, $\phi = \pi/3$.

In [7]:
# Step 7: Sample derivatives and Christoffel symbols

rfm_sph = reference_metric["Spherical"]

# Pick some representative components symbolically:
det_r = rfm_sph.detgammahatdD[0]
det_th = rfm_sph.detgammahatdD[1]

Gamma_r_thth = rfm_sph.GammahatUDD[0][1][1]
Gamma_th_rth = rfm_sph.GammahatUDD[1][0][1]

print("Sample first derivatives of detgammahat:")
print(" d/dx0 detgammahat:", det_r)
print(" d/dx1 detgammahat:", det_th)

print("\nSample Christoffel symbols for Spherical coordinates:")
print(" Gamma^r_{theta theta}:", Gamma_r_thth)
print(" Gamma^theta_{r theta}:", Gamma_th_rth)

# Evaluate these expressions at a concrete point, e.g. r = 2, theta = pi/4, phi = pi/3.
subs_point = {
    rfm_sph.xx[0]: sp.Integer(2),
    rfm_sph.xx[1]: sp.pi / 4,
    rfm_sph.xx[2]: sp.pi / 3,
}

print("\nNumerical evaluation at r=2, theta=pi/4, phi=pi/3:")
print(" d/dx0 detgammahat:", sp.N(det_r.subs(subs_point)))
print(" d/dx1 detgammahat:", sp.N(det_th.subs(subs_point)))
print(" Gamma^r_{theta theta}:", sp.N(Gamma_r_thth.subs(subs_point)))
print(" Gamma^theta_{r theta}:", sp.N(Gamma_th_rth.subs(subs_point)))

Sample first derivatives of detgammahat:
 d/dx0 detgammahat: 4*xx0**3*sin(xx1)**2
 d/dx1 detgammahat: 2*xx0**4*sin(xx1)*cos(xx1)

Sample Christoffel symbols for Spherical coordinates:
 Gamma^r_{theta theta}: -xx0
 Gamma^theta_{r theta}: 1/xx0

Numerical evaluation at r=2, theta=pi/4, phi=pi/3:
 d/dx0 detgammahat: 16.0000000000000
 d/dx1 detgammahat: 16.0000000000000
 Gamma^r_{theta theta}: -2.00000000000000
 Gamma^theta_{r theta}: 0.500000000000000


# Step 8: Precomputing expensive functions with _rfm_precompute
### \[Back to [top](#Table-of-Contents)\]

Some coordinate systems use transcendental functions (for example sinh or tanh profiles) in their definitions. Evaluating these inside tight finite difference loops can be expensive.

To address this, `ReferenceMetric` supports an "rfm precompute" mode. For each coordinate system `"Name"`, the cache also stores a companion `"Name_rfm_precompute"` object constructed with `enable_rfm_precompute=True`.

In this mode:

* Expressions such as the metric, its determinant, and rescaling vectors are rewritten in terms of abstract one dimensional function symbols like `f0_of_xx0` and their derivatives.
* These symbols are designed to be precomputed as grid functions and stored in a small struct on the C side.
* Derivatives that are identically zero are simplified away, so constant pieces do not cost any runtime.

Let us compare the `"SinhSpherical"` coordinate system with and without precomputation enabled.

In [8]:
# Step 8: Compare plain and _rfm_precompute variants for SinhSpherical

rfm_sinh_plain = reference_metric["SinhSpherical"]
rfm_sinh_pre = reference_metric["SinhSpherical_rfm_precompute"]

print("detgammahat without rfm_precompute:")
print(rfm_sinh_plain.detgammahat)

print("\ndetgammahat with rfm_precompute (expressed in terms of funcform symbols):")
print(rfm_sinh_pre.detgammahat)

print("\nExample rescaling factor ReU[0] without rfm_precompute:")
print(rfm_sinh_plain.ReU[0])

print("\nExample rescaling factor ReU[0] with rfm_precompute:")
print(rfm_sinh_pre.ReU[0])

print("\nIndependent funcform symbols that survived simplification:")
print(rfm_sinh_pre.freevars_uniq_xx_indep)

Setting up reference_metric[SinhSpherical]...
detgammahat without rfm_precompute:
AMPL**6*(exp(xx0/SINHW)/SINHW + exp(-xx0/SINHW)/SINHW)**2*(exp(xx0/SINHW) - exp(-xx0/SINHW))**4*sin(xx1)**2/(exp(1/SINHW) - exp(-1/SINHW))**6

detgammahat with rfm_precompute (expressed in terms of funcform symbols):
f0_of_xx0**4*f0_of_xx0__D0**2*f1_of_xx1**2

Example rescaling factor ReU[0] without rfm_precompute:
(exp(1/SINHW) - exp(-1/SINHW))/(AMPL*(exp(xx0/SINHW)/SINHW + exp(-xx0/SINHW)/SINHW))

Example rescaling factor ReU[0] with rfm_precompute:
1/f0_of_xx0__D0

Independent funcform symbols that survived simplification:
[f0_of_xx0__D0, f1_of_xx1, f0_of_xx0, f0_of_xx0__DD00, f0_of_xx0__DDD000, f1_of_xx1__D1, f1_of_xx1__DD11]


# Step 9: Advanced coordinate maps: sinh families, NewtonRaphson inversions, and validation
### \[Back to [top](#Table-of-Contents)\]

Finally, let us briefly explore some of the more advanced coordinate systems supported by `ReferenceMetric`.

1. **Sinh families.**
   Coordinate systems whose names start with `"Sinh"` use hyperbolic sine profiles to stretch the grid. The helper methods `Sinhv1` and `Sinhv2` implement these one dimensional maps. For example, `"SinhSpherical"` uses `Sinhv1` for the radial coordinate, while `"SinhSphericalv2n2"` uses `Sinhv2` with additional control over the small radius slope.

2. **NewtonRaphson inversions.**
   Some coordinate maps have simple forward formulas but no closed form inverse. In these cases the corresponding `ReferenceMetric` sets `requires_NewtonRaphson_for_Cart_to_xx=True` and fills the array `NewtonRaphson_f_of_xx` with the scalar equation whose root yields the appropriate coordinate value.

3. **Validation helpers.**
   The exported lists `supported_CoordSystems` and `unittest_CoordSystems` are used by standalone validation scripts to confirm that each coordinate system inverts correctly and that reference metric expressions remain stable as SymPy evolves.

The examples below show how to inspect these features from Python.

In [9]:
# Step 9: Inspect sinh-based maps and Newton-Raphson metadata

# Example 9a: Look at the one-dimensional map used in SinhSphericalv2n2.
rfm_sinh_v2 = reference_metric["SinhSphericalv2n2"]

x = sp.symbols("x", real=True)
AMPL, SINHW, slope = sp.symbols("AMPL SINHW slope", real=True)

sinhv1_expr = rfm_sinh_v2.Sinhv1(x, AMPL, SINHW)
sinhv2_expr = rfm_sinh_v2.Sinhv2(x, AMPL, SINHW, slope)

print("Sinhv1(x, AMPL, SINHW):")
print(sinhv1_expr)

print("\nSinhv2(x, AMPL, SINHW, slope) for the CoordSystem", rfm_sinh_v2.CoordSystem, ":")
print(sinhv2_expr)

# Example 9b: Coordinate systems that require Newton-Raphson inversions.
print("\nCoordSystems that require Newton-Raphson for Cart_to_xx:")
for name in rfm.supported_CoordSystems:
    r = reference_metric[name]
    if getattr(r, "requires_NewtonRaphson_for_Cart_to_xx", False):
        print(" -", name)

# Pick one and view its root-finding equation.
rfm_nr = reference_metric["SinhSphericalv2n2"]
print("\nNewtonRaphson_f_of_xx for", rfm_nr.CoordSystem, ":")
print(rfm_nr.NewtonRaphson_f_of_xx)

# Example 9c: Small summary of all CoordSystems, focusing on a few key flags.
print("\nSummary of supported coordinate systems:")
for name in rfm.supported_CoordSystems:
    r = reference_metric[name]
    print(
        " - {0:25s} | EigenCoord={1:10s} | requires_NR={2}".format(
            name,
            r.EigenCoord,
            getattr(r, "requires_NewtonRaphson_for_Cart_to_xx", False),
        )
    )

Setting up reference_metric[SinhSphericalv2n2]...
Sinhv1(x, AMPL, SINHW):
AMPL*(exp(x/SINHW) - exp(-x/SINHW))/(exp(1/SINHW) - exp(-1/SINHW))

Sinhv2(x, AMPL, SINHW, slope) for the CoordSystem SinhSphericalv2n2 :
slope*x + x**2*(AMPL - slope)*(exp(x/SINHW) - exp(-x/SINHW))/(exp(1/SINHW) - exp(-1/SINHW))

CoordSystems that require Newton-Raphson for Cart_to_xx:
 - SinhSphericalv2n2
Setting up reference_metric[SinhCartesian]...
Setting up reference_metric[Cylindrical]...
Setting up reference_metric[SinhCylindrical]...
Setting up reference_metric[SinhCylindricalv2n2]...
 - SinhCylindricalv2n2
Setting up reference_metric[SymTP]...
Setting up reference_metric[SinhSymTP]...
Setting up reference_metric[LWedgeHSinhSph]...
Setting up reference_metric[UWedgeHSinhSph]...
Setting up reference_metric[RingHoleySinhSpherical]...
Setting up reference_metric[HoleySinhSpherical]...

NewtonRaphson_f_of_xx for SinhSphericalv2n2 :
[r_slope*xx0 + xx0**2*(AMPL - r_slope)*(exp(xx0/SINHW) - exp(-xx0/SINHW))/(ex